In [ ]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
from knockouts import *

In [ ]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [ ]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/tmp.log", filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [ ]:
exchanges = ["EX_ac(e)","EX_but(e)","EX_ppa(e)"]

In [ ]:
from overflow import get_overflow
def run_context_specific(model):
    org = model.id
    ListHandler.print_and_log("Running {}".format(org))
    index_dct = {r.id : model.reactions.index(r) for r in model.reactions}
    met_index_dct = {m.id : model.metabolites.index(m) for m in model.metabolites}
    me_nlp = get_nlp(model)
    max_mu = model.solution.objective_value*2
    del model
    # Run for all conditions
    killable = pandas.read_csv("./ibdmdb/integration/{}/{}_killable.txt".format("IBD",org),header=None,index_col=0).index.to_list()
    for k in killable:
        knockout(index_dct,k,me_nlp,limit=lambda x:0)
    sol = optimize(index_dct,met_index_dct,me_nlp,max_mu=max_mu)
    ListHandler.print_and_log("Solution for {} : {}".format(org, sol))
    return sol

In [ ]:
from diets import load_model,constrain_exchanges,get_bounds
from overflow import get_overflow
import coralme

conditions = ["high"]
metabolite = "FA"

for c in conditions:
    name = "{}_{}".format(c,metabolite)
    fluxdir = "./cases/fluxes/{}-context".format(name)
    if not os.path.isdir(fluxdir):
        os.makedirs(fluxdir)

def run(org):
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    model.id = org
    sol0 = model.solution
    # Exchanges
    rlist = [r for r in exchanges if r in model.reactions]
    # Run for all conditions
    for c in conditions:
        name = "{}_{}".format(c,metabolite)
        dct = {k:-2.0 for k in rlist if k in model.reactions}
        constrain_exchanges(model,dct)
        sol = run_context_specific(model)
        sol.to_frame().to_csv("cases/fluxes/{}-context/{}.csv".format(name,org))

In [ ]:
# !bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
run_for = organisms
len(run_for)

In [ ]:
NP = min([4,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)

for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()